In [54]:
import os
import numpy as np
import pandas as pd

In [55]:
# Columns to keep.
usecols = [
    'Application Id',
    'Utility',
    'Service City',
    'Service Zip',
    'Service County',
    'Technology Type',
    'System Size DC',
    'System Size AC',
    'Inverter Size (kW AC)',
    'Tilt',
    'Azimuth',
    'Mounting Method',
    'Tracking',
    'Customer Sector',
    'App Approved Date',
    'Total System Cost',
    'Itc Cost Basis',
    'NEM Tariff',
    'Interconnection Program',
    'VNEM, NEM-V, NEM-Agg',
    'Project is VNEM, NEM-V, NEM-Agg?',
    'NEMPV or nonNEMPV'
]

# Utilities. 
utilities = [
    'PGE',
    'SDGE',
    'SCE'
]

# Technologies in which solar is the sole means of generation.
# NOTE: should we include sites with storage?
technologies = [
    'Solar PV',
    'Solar',
    'Solar PV, Storage',
    'Solar PV;Storage',
    'SOLAR PV',
    'Other, Solar PV',
    'Other, Solar PV, Storage'
]

In [56]:
#--- LOAD ---#

# Directory path. 
dir = 'data/Interconnected_Project_Sites_2023-03-31'

# Dataframe for sites. 
df = pd.DataFrame()

# Combine interconnection data from all utilities.
for file in os.listdir(dir):

    # Update path.
    path = os.path.join(dir, file)

    # Read data from one utility into dataframe. 
    subset = pd.read_csv(path, usecols=usecols)

    # Append. 
    df = pd.concat([df, subset])

/var/folders/zw/znrrh10n18z0wf8gggwxj0jc0000gn/T/ipykernel_57267/1725909909.py:16: DtypeWarning: Columns (13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  subset = pd.read_csv(path, usecols=usecols)
/var/folders/zw/znrrh10n18z0wf8gggwxj0jc0000gn/T/ipykernel_57267/1725909909.py:16: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  subset = pd.read_csv(path, usecols=usecols)
/var/folders/zw/znrrh10n18z0wf8gggwxj0jc0000gn/T/ipykernel_57267/1725909909.py:16: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  subset = pd.read_csv(path, usecols=usecols)


In [57]:
#--- FILTER ---#

# Filter data by utility, customer sector, tariff structure, and technology type.
df = df.loc[
    (df['Utility'].isin(utilities)) &
    (df['Customer Sector'] == 'Residential') &
    (df['NEMPV or nonNEMPV'] == 'NEMPV') & 
    (df['Technology Type'].isin(technologies))
]

In [58]:
#--- CLEAN ---#

# Convert to datetime.
df['App Approved Date'] = pd.to_datetime(df['App Approved Date'])

# Sort by date.
df.sort_values(by=['App Approved Date'], ascending=False, inplace=True, ignore_index=True)

# Convert ZIP codes to strings.
df['Service Zip'] = df['Service Zip'].astype(str).str.zfill(5).str.slice(0, 5)

# Convert tariff types to strings.
df['NEM Tariff'] = df['NEM Tariff'].astype(str)

# Convert strings to uppercase for nonnumeric columns.
df = df.apply(lambda x: x.str.upper() if x.dtype == "object" else x)

# Make labels compatible across utilities.
df.loc[df['Technology Type'] == 'SOLAR', ['Technology Type']] = 'SOLAR PV'
df.loc[df['Technology Type'] == 'OTHER, SOLAR PV', ['Technology Type']] = 'SOLAR PV'
df.loc[df['Technology Type'] == 'SOLAR PV;STORAGE', ['Technology Type']] = 'SOLAR PV, STORAGE'
df.loc[df['Technology Type'] == 'OTHER, SOLAR PV, STORAGE', ['Technology Type']] = 'SOLAR PV, STORAGE'
df.loc[df['Mounting Method'] == 'MIXED', ['Mounting Method']] = 'MULTIPLE'
df.loc[df['Tracking'] == 'MIXED', ['Tracking']] = 'MULTIPLE'
df.loc[df['Tracking'] == 'TRACKING', ['Tracking']] = 'SINGLE-AXIS'

In [59]:
#--- ASSUMPTIONS ---#

# If no tilt or azimuth is specified, assign the most common value.
df['Tilt'].fillna(value=df['Tilt'].value_counts().index[0], inplace=True) # 18.0
df['Azimuth'].fillna(value=df['Azimuth'].value_counts().index[0], inplace=True) # 180.0

# For sites with multiple tilts or azimuths, assign most common value.
df.loc[df['Tilt'] == 'MULTIPLE', ['Tilt']] = df['Tilt'].value_counts().index[0] # 18.0
df.loc[df['Azimuth'] == 'MULTIPLE', ['Azimuth']] = df['Azimuth'].value_counts().index[0] # 180.0

# Convert tilt and azimuth to floats (and assign NaNs for tracking systems).
df['Tilt'] = pd.to_numeric(df['Tilt'], errors='coerce')
df['Azimuth'] = pd.to_numeric(df['Azimuth'], errors='coerce')

# If no mounting method or tracking style is specified, assign the most common value.
df['Mounting Method'].fillna(value=df['Mounting Method'].value_counts().index[0], inplace=True)
df['Tracking'].fillna(value=df['Tracking'].value_counts().index[0], inplace=True)

# Replace instances of zero cost with NaNs.
df.loc[df['Total System Cost'] == 0.0, ['Total System Cost']] = float('NaN')
df.loc[df['Itc Cost Basis'] == 0.0, ['Itc Cost Basis']] = float('NaN')

# Correct what are presumably erroneous negative system and inverter sizes.
df['System Size DC'] = df['System Size DC'].abs()
df['System Size AC'] = df['System Size AC'].abs()
df['Inverter Size (kW AC)'] = df['Inverter Size (kW AC)'].abs()

In [60]:
#--- EXPORT ---#

# Write dataframe to CSV.
df.to_csv(f'{dir}/Aggregated_Sites.csv', index=False)

In [61]:
#--- VALIDATE ---#

# Describe categorical variables.
df.describe(include=object)

,Application Id,Utility,Service City,Service Zip,Service County,Technology Type,Mounting Method,Tracking,Customer Sector,NEM Tariff,Interconnection Program,"VNEM, NEM-V, NEM-Agg","Project is VNEM, NEM-V, NEM-Agg?",NEMPV or nonNEMPV
count,1500393,1500393,1500393,1500393,1500393,1500393,1500393,1500393,1500393,1500393,1500393,1500393,1500393,1500393
unique,1500393,3,1182,1646,54,2,3,4,1,2,39,4,2,1
top,SCE-INT-NST-396139,PGE,SAN DIEGO,95648,SAN DIEGO,SOLAR PV,ROOFTOP,FIXED,RESIDENTIAL,2.0,SNEM,NONE,NO,NEMPV
freq,1,700084,79179,10237,235666,1437192,1476832,1493433,1500393,963671,656767,1492022,1492022,1500393


In [62]:
# Check values.
df['Technology Type'].value_counts(dropna=False)

SOLAR PV             1437192
SOLAR PV, STORAGE      63201
Name: Technology Type, dtype: int64

In [63]:
# Check values.
df['Mounting Method'].value_counts(dropna=False)

ROOFTOP     1476832
GROUND        21654
MULTIPLE       1907
Name: Mounting Method, dtype: int64

In [64]:
# Check values.
df['Tracking'].value_counts(dropna=False)

FIXED          1493433
SINGLE-AXIS       3828
MULTIPLE          2249
DUAL-AXIS          883
Name: Tracking, dtype: int64

In [65]:
# Check values. 
df['Project is VNEM, NEM-V, NEM-Agg?'].value_counts(dropna=False)

NO     1492022
YES       8371
Name: Project is VNEM, NEM-V, NEM-Agg?, dtype: int64

In [66]:
# Check values. 
df['VNEM, NEM-V, NEM-Agg'].value_counts(dropna=False)

NONE       1492022
NEM-AGG       7211
VNEM           919
NEM-V          241
Name: VNEM, NEM-V, NEM-Agg, dtype: int64

In [67]:
# Describe numeric variables.
df.describe(include=np.number)

,System Size DC,System Size AC,Inverter Size (kW AC),Tilt,Azimuth,Total System Cost,Itc Cost Basis
count,1.500393e+06,1.500393e+06,946132.000000,1.497892e+06,1.497889e+06,8.030550e+05,3.934470e+05
mean,6.412854e+00,6.033560e+00,5.171258,1.842286e+01,1.798956e+02,2.900748e+04,2.430622e+04
std,5.444554e+00,5.141241e+00,13.589377,6.935201e+00,6.459484e+01,2.351536e+04,1.916353e+04
min,0.000000e+00,2.000000e-03,0.000000,0.000000e+00,0.000000e+00,1.000000e-02,1.000000e-02
25%,4.015000e+00,3.779000e+00,3.000000,1.800000e+01,1.730000e+02,1.785000e+04,1.365440e+04
50%,5.670000e+00,5.320000e+00,4.600000,1.800000e+01,1.800000e+02,2.528000e+04,2.128950e+04
75%,7.920000e+00,7.459000e+00,7.200000,2.100000e+01,2.060000e+02,3.548500e+04,3.175000e+04
max,1.106560e+03,9.988000e+02,8149.412000,3.580000e+02,3.600000e+02,3.129000e+06,2.271734e+06
